In [89]:
!pip install tensorflow-model-optimization

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [90]:
import keras
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os
import sys

In [91]:
base_path = "/s/chopin/k/grad/C837304112/Documents/ESML/final_project/human_activity_recognition"
base_path

'/s/chopin/k/grad/C837304112/Documents/ESML/final_project/human_activity_recognition'

In [92]:
uci_har_dataset_path = f'{base_path}/data/UCI_HAR'
uci_har_dataset_path

'/s/chopin/k/grad/C837304112/Documents/ESML/final_project/human_activity_recognition/data/UCI_HAR'

In [93]:
if base_path not in sys.path:
    sys.path.append(base_path) 

In [94]:
import src.load_uci_har_dataset as uci_har

In [95]:
Xtrain, ytrain, xtest, ytest, labels = uci_har.load_uci_har_dir(uci_har_dataset_path)

In [96]:
Xtrain.shape, ytrain.shape, xtest.shape, ytest.shape, labels,len(labels)

((7352, 128, 6),
 (7352,),
 (2947, 128, 6),
 (2947,),
 ['WALKING',
  'WALKING_UPSTAIRS',
  'WALKING_DOWNSTAIRS',
  'SITTING',
  'STANDING',
  'LAYING'],
 6)

In [97]:
import src.get_models as models

In [98]:
input_shape = Xtrain.shape[1:]
num_classes = len(labels)
input_shape, num_classes

((128, 6), 6)

In [99]:
import importlib
importlib.reload(models)

<module 'src.get_models' from '/s/chopin/k/grad/C837304112/Documents/ESML/final_project/human_activity_recognition/src/get_models.py'>

In [100]:
base_model = keras.models.load_model(f'{base_path}/models/cnn_lstm_model.keras')

2025-12-15 17:46:20.741158: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-12-15 17:46:20.742093: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-12-15 17:46:20.742723: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [101]:
base_model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 128, 6)]          0         
                                                                 
 conv1d_4 (Conv1D)           (None, 128, 64)           1984      
                                                                 
 batch_normalization_6 (Batc  (None, 128, 64)          256       
 hNormalization)                                                 
                                                                 
 re_lu_4 (ReLU)              (None, 128, 64)           0         
                                                                 
 conv1d_5 (Conv1D)           (None, 128, 128)          24704     
                                                                 
 batch_normalization_7 (Batc  (None, 128, 128)         512       
 hNormalization)                                           

In [102]:
baseline_model_loss, baseline_model_accuracy = base_model.evaluate(
    xtest, ytest)

print('Baseline test accuracy:', baseline_model_accuracy)


2025-12-15 17:46:27.199423: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-12-15 17:46:27.199978: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-12-15 17:46:27.200738: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

93/93 [==============================] - 6s 58ms/step - loss: 0.2449 - accuracy: 0.9321
Baseline test accuracy: 0.9321343898773193


In [103]:
q_aware_model = models.get_qat_model_dense_only(input_shape, num_classes)

2025-12-15 17:46:36.248193: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-12-15 17:46:36.248815: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-12-15 17:46:36.249496: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2025-12-15 17:46:37.088235: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-12-15 17:46:37.088843: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-12-15 17:46:37.089479: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [104]:
q_aware_model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3),
          loss='sparse_categorical_crossentropy',
          metrics=['accuracy'])

In [105]:
q_aware_model.summary()

Model: "model_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_15 (InputLayer)       [(None, 128, 6)]          0         
                                                                 
 conv1d_27 (Conv1D)          (None, 128, 64)           1984      
                                                                 
 batch_normalization_39 (Bat  (None, 128, 64)          256       
 chNormalization)                                                
                                                                 
 re_lu_26 (ReLU)             (None, 128, 64)           0         
                                                                 
 conv1d_28 (Conv1D)          (None, 128, 128)          24704     
                                                                 
 batch_normalization_40 (Bat  (None, 128, 128)         512       
 chNormalization)                                         

In [116]:
lr_decay_callback = keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5,
                                      patience=8, min_lr=1e-6, verbose=1)
early_stop = keras.callbacks.EarlyStopping(monitor="val_loss", patience=20, restore_best_weights=True)
q_aware_model.fit(Xtrain, ytrain, epochs=150, batch_size=64,
                          validation_split=0.1, verbose=1, callbacks=[early_stop, lr_decay_callback])

Epoch 1/150
104/104 [==============================] - 10s 95ms/step - loss: 0.1497 - accuracy: 0.9436 - val_loss: 0.0919 - val_accuracy: 0.9755 - lr: 1.0000e-06
Epoch 2/150
104/104 [==============================] - 10s 96ms/step - loss: 0.1502 - accuracy: 0.9414 - val_loss: 0.0984 - val_accuracy: 0.9715 - lr: 1.0000e-06
Epoch 3/150
104/104 [==============================] - 10s 96ms/step - loss: 0.1518 - accuracy: 0.9424 - val_loss: 0.0913 - val_accuracy: 0.9728 - lr: 1.0000e-06
Epoch 4/150
104/104 [==============================] - 10s 97ms/step - loss: 0.1459 - accuracy: 0.9438 - val_loss: 0.0920 - val_accuracy: 0.9742 - lr: 1.0000e-06
Epoch 5/150
104/104 [==============================] - 10s 96ms/step - loss: 0.1520 - accuracy: 0.9409 - val_loss: 0.0952 - val_accuracy: 0.9715 - lr: 1.0000e-06
Epoch 6/150
104/104 [==============================] - 10s 97ms/step - loss: 0.1500 - accuracy: 0.9424 - val_loss: 0.0907 - val_accuracy: 0.9715 - lr: 1.0000e-06
Epoch 7/150
104/104 [=======

In [117]:
q_aware_model_loss, q_aware_model_accuracy = q_aware_model.evaluate(
   xtest, ytest)

print('Quant test accuracy:', q_aware_model_accuracy)

93/93 [==============================] - 3s 29ms/step - loss: 0.2327 - accuracy: 0.9260
Quant test accuracy: 0.926026463508606


In [118]:
import tensorflow as tf
run_model = tf.function(lambda x: q_aware_model(x))
BATCH_SIZE = 1
STEPS = 128
INPUT_SIZE = 6
concrete_func = run_model.get_concrete_function(
    tf.TensorSpec([BATCH_SIZE, STEPS, INPUT_SIZE], q_aware_model.inputs[0].dtype))

# model directory.
MODEL_DIR = f"{base_path}/LSTM_CNN_QAT"
q_aware_model.save(MODEL_DIR, save_format="tf", signatures=concrete_func)

2025-12-15 18:37:32.102773: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-12-15 18:37:32.103814: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-12-15 18:37:32.104439: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2025-12-15 18:37:33.087705: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-12-15 18:37:33.088534: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-12-15 18:37:33.089060: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2025-12-15 18:37:34.042033: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-12-15 18:37:34.042913: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-12-15 18:37:34.043458: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2025-12-15 18:37:34.756981: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-12-15 18:37:34.757816: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-12-15 18:37:34.758365: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

INFO:tensorflow:Assets written to: /s/chopin/k/grad/C837304112/Documents/ESML/final_project/human_activity_recognition/LSTM_CNN_QAT/assets


INFO:tensorflow:Assets written to: /s/chopin/k/grad/C837304112/Documents/ESML/final_project/human_activity_recognition/LSTM_CNN_QAT/assets


In [119]:
converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_DIR)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

quantized_tflite_model = converter.convert()

2025-12-15 18:37:36.373381: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_2_grad_concat_split_2_split_dim' with dtype int32
	 [[{{node gradients_split_2_grad_concat_split_2_split_dim}}]]
2025-12-15 18:37:36.373456: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_grad_concat_split_split_dim' with dtype int32
	 [[{{node gradients_split_grad_concat_split_split_dim}}]]
2025-12-15 18:37:36.373492: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2025-12-15 18:37:36.857059: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_2_grad_concat_split_2_split_dim' with dtype int32
	 [[{{node gradients_split_2_grad_concat_split_2_split_dim}}]]
2025-12-15 18:37:36.857134: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_grad_concat_split_split_dim' with dtype int32
	 [[{{node gradients_split_grad_concat_split_split_dim}}]]
2025-12-15 18:37:36.857168: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2025-12-15 18:37:37.746320: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2025-12-15 18:37:37.746347: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2025-12-15 18:37:37.746957: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /s/chopin/k/grad/C837304112/Documents/ESML/final_project/human_activity_recognition/LSTM_CNN_QAT
2025-12-15 18:37:37.761472: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2025-12-15 18:37:37.761500: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /s/chopin/k/grad/C837304112/Documents/ESML/final_project/human_activity_recognition/LSTM_CNN_QAT
2025-12-15 18:37:37.815114: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2025-12-15 18:37:37.941859: I tensorflow/cc/saved_model/loader.cc:215] Running initialization op on SavedModel bundle at pat

In [120]:
with open(f"{base_path}/models/cnn_lstm_qat.tflite", "wb") as file:
    file.write(quantized_tflite_model)

In [121]:
def evaluate_model(interpreter):
    in_det  = interpreter.get_input_details()[0]
    out_det = interpreter.get_output_details()[0]
    in_idx  = in_det["index"]
    out_idx = out_det["index"]
    in_dtype = in_det["dtype"]     
    in_scale, in_zero = in_det["quantization"]
    print("dtype", in_dtype)

    prediction_digits = []
    for i, test_image in enumerate(xtest):
        if i % 1000 == 0:
            print(f"Evaluated on {i} results so far.")

        xi = np.expand_dims(test_image, axis=0).astype(np.float32)

        if in_dtype == np.int8:
            xi_q = np.clip(np.round(xi / in_scale + in_zero), -128, 127).astype(np.int8)
            interpreter.set_tensor(in_idx, xi_q)
        elif in_dtype == np.uint8:
            xi_q = np.round(xi / in_scale + in_zero).astype(np.uint8)
            interpreter.set_tensor(in_idx, xi_q)
        else:
            interpreter.set_tensor(in_idx, xi)

        interpreter.invoke()

        yo = interpreter.get_tensor(out_det["index"])[0]
        prediction_digits.append(int(np.argmax(yo)))

    print('\n')
    prediction_digits = np.array(prediction_digits)
    accuracy = (prediction_digits == ytest).mean()
    return accuracy

In [122]:
interpreter = tf.lite.Interpreter(model_path=f"{base_path}/models/cnn_lstm_qat.tflite")
interpreter.allocate_tensors()

test_accuracy_qat_tflite = evaluate_model(interpreter)
print("quant aware training model accuracy", test_accuracy_qat_tflite)

dtype <class 'numpy.float32'>
Evaluated on 0 results so far.
Evaluated on 1000 results so far.
Evaluated on 2000 results so far.


quant aware training model accuracy 0.9297590770274856


In [124]:
print("CNN-LSTM QAT model size in MB:", os.path.getsize(f"{base_path}/models/cnn_lstm_qat.tflite") / float(2**20))

CNN-LSTM QAT model size in MB: 0.8375396728515625
